# Antenas y Mediciones

In [1]:
# LIBRERIAS
from pykml import parser
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline

# CARGAR ARCHIVO
filename='RNI.kml'
with open(filename) as f:
    folder = parser.parse(f).getroot().Document.Folder

In [6]:
# EXTRACCION DE DATOS

# Definicion de Dataframes
Antenas     = pd.DataFrame()
Mediciones1 = pd.DataFrame()
Mediciones2 = pd.DataFrame()


# Listas auxiliares [valor, coordenadas]
#Ant  = [[], []]
#Med1 = [[], []]
#Med2 = [[], []]



# Funcion extractora de datos en formato KML
def data_extractor(data):
    aux = ''
    for i in range(5):
        if data[i+5]=='<':
            break
        else:
            aux=aux+data[i+5]  
    return float(aux)

# Algoritmo de carga de las listas auxiliares
def data_upload(folder):
    #global Ant, Med1, Med2
    aux =[[],[],[],[],[],[]]
    for pm in folder.Placemark:
        plnm1=str(pm.name)
        plcs1=str(pm.Point.coordinates)
        if plnm1.startswith('A',0) or plnm1.startswith('T',0):
            data=str(pm.description)
            aux[0].append(data_extractor(data))
            aux[1].append(plcs1)
        elif plnm1.startswith('M',0):
            data=str(pm.description)
            aux[2].append(data_extractor(data))
            aux[3].append(plcs1)
        elif plnm1.startswith('H',0):
            data=str(pm.description)
            aux[4].append(data_extractor(data))
            aux[5].append(plcs1)
    return aux 
        

def send_dataframes(i,db):
    aux = pd.DataFrame()
    if i==0:
        aux['altura']      = db[0]
        aux['coordenadas'] = db[1]
    elif i==1:
        aux['densidad de potencia']      = db[2]
        aux['coordenadas'] = db[3]
    elif i==2:
        aux['densidad de potencia']      = db[4]
        aux['coordenadas'] = db[5]
    aux['Longitude'],aux['Latitude'],aux['value']=zip(*aux['coordenadas'].apply(lambda x: x.split(',',2)))
    return aux
    
db = data_upload(folder)

Antenas     = send_dataframes(0,db)
Mediciones1 = send_dataframes(1,db)
Mediciones2 = send_dataframes(2,db)
    
# Carga de datos en Dataframes
#Antenas['altura']      = Ant[0]
#Antenas['coordenadas'] = Ant[1]

#Mediciones1['densidad de potencia'] = Med1[0]
#Mediciones1['coordenadas']          = Med1[1]
#Mediciones2['densidad de potencia'] = Med2[0]
#Mediciones2['coordenadas']          = Med2[1]

# Conversion de coordenadas a Longitud y Latitud
#Antenas['Longitude'],Antenas['Latitude'],Antenas['value']=zip(*Antenas['coordenadas'].apply(lambda x: x.split(',',2)))
#Mediciones1['Longitude'],Mediciones1['Latitude'],Mediciones1['value']=zip(*Mediciones1['coordenadas'].apply(lambda x: x.split(',',2)))
#Mediciones2['Longitude'],Mediciones2['Latitude'],Mediciones2['value']=zip(*Mediciones2['coordenadas'].apply(lambda x: x.split(',',2)))

In [9]:
Mediciones2

,densidad de potencia,coordenadas,Longitude,Latitude,value
0,2.006,"-57.54265074968813,-38.00946881020497,22.20418...",-57.54265074968813,-38.00946881020497,22.20418645483292
1,1.456,"-57.54400435251471,-38.00914892090206,24.35478...",-57.54400435251471,-38.00914892090206,24.35478199461131
2,14.870,"-57.54454901544158,-38.00793373672784,28.58025...",-57.54454901544158,-38.00793373672784,28.58025967589007
3,0.319,"-57.58132540226884,-38.01197453195897,19.31765...",-57.58132540226884,-38.01197453195897,19.31765706776068


In [ ]:
#Antenas['altura'].describe()
Antenas["altura"].plot.density(color="blueviolet").set_xlim(10,135)

In [ ]:
Mediciones3 = Mediciones1
Mediciones3.drop(59,axis=0,inplace=True)
#Mediciones3["densidad de potencia"].describe()
#Mediciones3["densidad de potencia"].plot.hist(bins=40,color="blueviolet").set_xlim(0,5.5)
Mediciones3["densidad de potencia"].plot.density(color="blueviolet").set_xlim(0,5)

In [ ]:
#Mediciones2["densidad de potencia"].describe()
Mediciones2["densidad de potencia"].plot.density(color="blueviolet").set_xlim(0,15)

In [ ]:
# GUARDAR EN BASE DE DATOS
from datetime import datetime
now = datetime.now()
fecha = str(now.day) + "_" + str(now.month) + "_" + str(now.year)

aux2 = ["Antenas_","Mediciones1_","Mediciones2_",".csv"]
for i in range(0,3):
    aux2[i] = aux2[i] + fecha + aux2[3]
    
Antenas.to_csv(aux2[0])
Mediciones1.to_csv(aux2[1])
Mediciones2.to_csv(aux2[2])

In [ ]:
import folium as fo
map1 = fo.Map(location=[-37.99,-57.55],
                    zoom_start = 14)
map2 = fo.Map(location=[-37.99,-57.55],
                    zoom_start = 14)
lat_fol_ant = list(Antenas["Latitude"].astype(float))
lon_fol_ant = list(Antenas["Longitude"].astype(float))
name_fol_ant = Antenas.index.tolist()
fol_antena = fo.FeatureGroup(name="Antenas")

lat_fol_med1 = list(Mediciones1["Latitude"].astype(float))
lon_fol_med1 = list(Mediciones1["Longitude"].astype(float))
name_fol_med1 = Mediciones1.index.tolist()
fol_medicion1 = fo.FeatureGroup(name="Mediciones1")

lat_fol_med2 = list(Mediciones2["Latitude"].astype(float))
lon_fol_med2 = list(Mediciones2["Longitude"].astype(float))
name_fol_med2 = Mediciones2.index.tolist()
fol_medicion2 = fo.FeatureGroup(name="Mediciones2")

In [ ]:
for lat,lon,name in zip(lat_fol_ant, lon_fol_ant,name_fol_ant):
    fol_antena.add_child(fo.Marker(location=[lat, lon], popup = name, icon = fo.Icon(color="blue",icon="wifi",prefix="fa")))
    
for lat1,lon1,name1 in zip(lat_fol_med1, lon_fol_med1,name_fol_med1):
                         fol_medicion1.add_child(fo.Marker(location=[lat1, lon1], popup = name1, icon = fo.Icon(color="green")))

for lat2,lon2,name2 in zip(lat_fol_med2, lon_fol_med2,name_fol_med2):
                         fol_medicion2.add_child(fo.Marker(location=[lat2, lon2], popup = name2, icon = fo.Icon(color="purple")))


In [ ]:
map1.add_child(fol_antena)
map1.add_child(fol_medicion1)
map1.add_child(fol_medicion2)

In [ ]:
import folium as fo
from folium import plugins
from folium.plugins import HeatMap

map35 = fo.Map(location=[-37.99,-57.55],
                    zoom_start = 14)

longs =Mediciones1["Longitude"]
lats = Mediciones1["Latitude"]

# calcular los valores medios de longitud y latitud
#import statistics
#meanLong = statistics.mean(longs)
#meanLat = statistics.mean(lats)

# crear capa de mapa de calor
heatmap = HeatMap(list(zip(lats, longs, Mediciones1["densidad de potencia"])),min_opacity=0.2,radius=30,gradient={0:
'blue',0.2: 'lime',0.7:'yellow',0.95: 'orange' ,1: 'red'},blur=5,max_zoom=2)
# añadir capa de mapa de calor al mapa base

heatmap.add_to(map35)
map35.add_child(fol_antena)
map35

In [ ]:
map35.add_child(fol_antena)

In [ ]:
# CALCULO DE DISTANCIA ENTRE PUNTOS
import math
import numpy as np
from numpy import pi, round

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def distancia_r(a,m):
    lat1 = float(Mediciones1["Latitude"][m])
    lon1 = float(Mediciones1["Longitude"][m])
    lat2 = float(Antenas["Latitude"][a])
    lon2 = float(Antenas["Longitude"][a])
    h    = float(Antenas["altura"][a])   
    R    = 6367*1000 # Radio de la tierra
    a    = math.sin(deg2rad(lat1)) * math.sin(deg2rad(lat2))
    b    = math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.cos(deg2rad(lon2 - lon1))
    c    = math.acos(a + b)
    d    = R * c   
    r    = np.sqrt(d**2 + h**2)
    return round(r)

def area(rm):
    rcm = rm/100  # m a cm
    return 4 * pi * (rcm**2)

In [ ]:
# CALCULO DE ESTIMA DE POTENCIA DE ANTENA POR MEDICION
def pot_dbm(a,m):
    r = distancia_r(a,m)
    a = area(r)
    s = float(Mediciones1['densidad de potencia'][m])
    pot = a * s  # mW
    pot_dbm = 10 * math.log10(pot)
    return float(pot_dbm)

In [ ]:
# Antena 43
antena_43 =[]
for i in range(135,140,1):
    antena_43.append(pot_dbm(43,i))
antena_43.sort()    
# Antena 44
antena_44 =[]
for i in range(140,144,1):
    antena_44.append(pot_dbm(44,i))
antena_44.sort()
# Antena 42
antena_42 =[]
for i in range(129,134,1):
    antena_42.append(pot_dbm(42,i))
antena_42.sort()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = [1,2,3,4,5]
y = antena_43 # 10, not 9, so the fit isn't perfect

coef = np.polyfit(x,y,1)
poly1d_fn = np.poly1d(coef) 
# poly1d_fn is now a function which takes in x and returns an estimate for y

plt.plot(x,y, 'ro', x, poly1d_fn(x), '--k') #'--k'=black dashed line, 'yo' = yellow circle marker

plt.xlim(1, 5)
plt.ylim(-5, 10)